In [2]:
import re
import spacy
import contractions
import joblib
import pandas as pd
import numpy as np
from datetime import datetime
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertTokenizer, BertModel
import torch
import nltk

In [3]:
new_df = pd.read_excel('/content/processed_data.xlsx')

In [4]:
# Use a single TfidfVectorizer for both Resume and Job Description
tfidf_vectorizer4 = TfidfVectorizer()
combined_text = new_df['Resume_processed'] + " " + new_df['Job_Description_processed']
tfidf_vectorizer4.fit(combined_text)

# Split the combined TF-IDF matrix back into Resume and Job Description matrices
resume_matrix = tfidf_vectorizer4.transform(new_df['Resume_processed'])
jobDescription_matrix = tfidf_vectorizer4.transform(new_df['Job_Description_processed'])

# Calculate row-wise cosine similarity
cos_rjd = []
for i in range(new_df.shape[0]):
    cos_rjd.append(cosine_similarity(resume_matrix[i], jobDescription_matrix[i])[0][0])

In [5]:
new_df['resume_jd_simi'] = cos_rjd

In [9]:
new_df.groupby(['Role' , 'decision'])['resume_jd_simi'].mean()

Role               decision
data analyst       reject      0.103219
                   select      0.076433
data engineer      reject      0.236726
                   select      0.236996
data scientist     reject      0.208495
                   select      0.202127
product manager    reject      0.309318
                   select      0.279775
software engineer  reject      0.188870
                   select      0.207007
ui designer        reject      0.103275
                   select      0.073463
ui engineer        reject      0.360938
                   select      0.377108
Name: resume_jd_simi, dtype: float64